## Python + MySQl

In [23]:
import mysql.connector
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# realizando una conexión con el gestor de bases de datos MySQL

conexion = mysql.connector.connect(host="localhost",                                             
                                   user="root",
                                   password="password",
                                   port="3306",
                                   database="northwind",
                                   auth_plugin="mysql_native_password")  

In [56]:
# realizando consultas que relacionan tablas y generan columnas nuevas

cursor_1 = conexion.cursor()

cursor_1.execute('''create view order_details_cash as
                    select OrderID, od.ProductID, Quantity, Price * Quantity as product_cash from OrderDetails od
                    join Products p on od.ProductID = p.ProductID;''')

cursor_1.execute('''create view products_revenue as
                    select ProductName, sum(product_cash) as product_revenue, sum(Quantity) as total_quantity from order_details_cash odc
                    join Products p on p.ProductID = odc.ProductID
                    group by p.ProductID;''')

cursor_1.execute('''select * from products_revenue where product_revenue > (select avg(product_revenue) from products_revenue)
                    order by product_revenue desc''')

# creando un Data frame que esté constituido por los datos provenientes de las consultas realizadas en la base de datos

data_1 = {
    "products":[],
    "products_revenue":[],
    "total_quantity":[]
}

for p in cursor_1:
    data_1["products"].append(p[0])
    data_1["products_revenue"].append(p[1])
    data_1["total_quantity"].append(p[2])


df_products = pd.DataFrame(data_1)

mean_products = df_products["products_revenue"].mean()
mean_products = mean_products.astype(str)
 
df_products # primer Data Frame (productos e ingresos)

,products,products_revenue,total_quantity
0,Côte de Blaye,63096,239
1,Thüringer Rostbratwurst,20832,168
2,Raclette Courdavault,19030,346
3,Tarte au sucre,15925,325
4,Camembert Pierrot,14620,430
5,Alice Mutton,12909,331
6,Gnocchi di nonna Alice,10222,269
7,Mozzarella di Giovanni,9450,270
8,Vegie-spread,9196,209
9,Rössle Sauerkraut,8694,189


In [57]:
cursor_2 = conexion.cursor()

cursor_2.execute('''create view orders_revenue as
                    select o.OrderID, sum(product_cash) as order_revenue, sum(Quantity) as total_quantity, EmployeeID from order_details_cash odc
                    join Orders o on o.OrderID = odc.OrderID
                    group by o.OrderID
                    order by order_revenue desc;''')

cursor_2.execute('''select * from orders_revenue where order_revenue > (select avg(order_revenue) from orders_revenue)''')

data_2 = {
    "orders_id":[],
    "orders_revenue":[],
    "total_quantity":[]
}

for o in cursor_2:
    data_2["orders_id"].append(o[0])
    data_2["orders_revenue"].append(o[1])
    data_2["total_quantity"].append(o[2])

df_orders = pd.DataFrame(data_2)

mean_orders = df_orders["orders_revenue"].mean()
mean_orders = mean_orders.astype(str)

df_orders["orders_id"] = df_orders["orders_id"].astype("string")

df_orders # segundo Data Frame (órdenes e ingresos)

,orders_id,orders_revenue,total_quantity
0,10372,15382,164
1,10424,14406,139
2,10417,14147,123
3,10353,13452,62
4,10360,9256,138
...,...,...,...
65,10346,2164,56
66,10373,2140,130
67,10404,2090,100
68,10312,2028,58


In [58]:
cursor_3 = conexion.cursor()

cursor_3.execute('''select concat (FirstName," ",LastName) as name, sum(order_revenue) as employee_revenue, sum(total_quantity) as total_quantity  from Employees e
                join orders_revenue o on o.EmployeeID = e.EmployeeID 
                group by e.EmployeeID
                order by employee_revenue''')

data_3 = {
    "names":[],
    "employees_revenue":[],
    "total_quantity":[]
}

for e in cursor_3:
    data_3["names"].append(e[0])
    data_3["employees_revenue"].append(e[1])
    data_3["total_quantity"].append(e[2])
    
df_employees = pd.DataFrame(data_3)

mean_employees = df_employees["employees_revenue"].mean()
mean_employees = mean_employees.astype(str)

df_employees # tercer y último Data Frame (empleados e ingresos)

,names,employees_revenue,total_quantity
0,Anne Dodsworth,15702,649
1,Michael Suyama,25501,1094
2,Steven Buchanan,27606,778
3,Andrew Fuller,32559,1315
4,Laura Callahan,39341,1293
5,Robert King,39843,733
6,Janet Leverling,42823,1725
7,Nancy Davolio,57765,1924
8,Margaret Peacock,105926,3232


#### Dashboard que refleja los principales componentes exitosos de ventas

In [ ]:
# Data Storytellyng

best_product = df_products.iloc[0,:] # Producto con el mayor ingreso de venta

# ---------------------------------------------------------------------------------------------------------

best_order = df_orders.iloc[0,:] # Órden con el mayor ingreso de venta

# ---------------------------------------------------------------------------------------------------------

df_employees.sort_values(by="employees_revenue", ascending=False, inplace=True)
best_employee = df_employees.iloc[0,:] # Empleado con el mayor ingreso de venta

# ---------------------------------------------------------------------------------------------------------


app = dash.Dash(__name__)

app.layout = html.Div(id="body",children=[
    html.Div(id="dropdown_div",className="e2_dropdown_div",children=[
            dcc.Dropdown(id="dropdown",className="e2_dropdown",
                        options = [
                            {"label":"Empleados","value":"names"},
                            {"label":"Productos","value":"products"},
                            {"label":"Órdenes","value":"orders_id"}
                        ],
                        value="names",
                        multi=False,
                        clearable=False)
    ]),
    dcc.Graph(id="graph",figure={}),
    html.H1("Mayores ingresos",className="e2_titulo"),
    html.Div(className="e2_container",children=[
        html.Div(id="data_1",className="e2_children",style={"color":"blue"},children=[   
            html.H2("Productos",style={"font-size":"1.15em","color":"blue","font-family":"sans-serif"}),
            html.P(f"Promedio: {mean_products[:6]}$",className="e2_mean",style={"color":"blue"}),         
            html.Ul(className="e2_ul",style={"color":"blue"},children=[
                html.Li(f"Producto: {best_product["products"]}",className="e2_list"),
                html.Li(f"Cantidad total vendida: {best_product["total_quantity"]}",className="e2_list"),
                html.Li(f"Ingresos totales: {best_product["products_revenue"]}$",className="e2_list")
            ])
        ]),
        html.Div(id="data_2",className="e2_children",children=[
            html.H2("Empleados",style={"font-size":"1.15em","color":"red","font-family":"sans-serif"}),
            html.P(f"Promedio: {mean_employees[:6]}$",className="e2_mean",style={"color":"red"}),
            html.Ul(className="e2_ul",style={"color":"red"},children=[
                html.Li(f"Nombre: {best_employee["names"]}",className="e2_list"),
                html.Li(f"Cantidad total vendida: {best_employee["total_quantity"]}",className="e2_list"),
                html.Li(f"Ingresos totales: {best_employee["employees_revenue"]}$",className="e2_list")
            ])
        ]),
        html.Div(id="data_3",className="e2_children",children=[
            html.H2("Órdenes",style={"font-size":"1.15em","color":"green","font-family":"sans-serif"}),
            html.P(f"Promedio: {mean_orders[:6]}$",className="e2_mean",style={"color":"green"}),
            html.Ul(className="e2_ul",style={"color":"green"},children=[
                html.Li(f"Órden: {best_order["orders_id"]}",className="e2_list"),
                html.Li(f"Cantidad total vendida: {best_order["total_quantity"]}",className="e2_list"),
                html.Li(f"Ingresos totales: {best_order["orders_revenue"]}$",className="e2_list")
            ])
        ]) 
    ])
])

@app.callback(
    Output(component_id="graph",component_property="figure"),
    [Input(component_id="dropdown",component_property="value")]
)

def update_graph(slct_data):
    
    if slct_data == "names":
        graph = px.bar(df_employees, x=slct_data, y="employees_revenue", color_discrete_sequence=['red'], text_auto='.2s', title="Ingresos de empleados",
                       labels=dict(names="Empleados", employees_revenue="Ingresos"))
    elif slct_data == "products":
        graph = px.bar(df_products, x=slct_data, y="products_revenue", color_discrete_sequence=['blue'], text_auto='.2s', title="Ingresos de productos mayores al promedio",
                       labels=dict(products="Productos", products_revenue="Ingresos"))
        graph.update_xaxes(tickangle=35, tickfont_size=8)
    elif slct_data == "orders_id":
        graph = px.bar(df_orders, x=slct_data, y="orders_revenue", color_discrete_sequence=['green'], title="Ingresos de órdenes mayores al promedio",
                       labels=dict(orders_id="Órdenes", orders_revenue="Ingresos"))
        graph.update_xaxes(tickfont_size=9)
    
    return graph    
    
if __name__ == "__main__":
    app.run_server(debug=False)
    
# eliminar las vistas creadas (opcional)

# delete_views = conexion.cursor()
# delete_views.execute('drop view products_revenue; drop view order_details_cash; drop view orders_revenue')